In [1]:
import random
from geoarray import GeoArray
import matplotlib.pyplot as plt
import numpy as np
import math
from scipy.signal import savgol_filter
from scipy.interpolate import splev, splrep

In [2]:
# importing ISOFIT
from isofit.utils import surface_model
from isofit.core.forward import ForwardModel
from isofit.configs.configs import create_new_config
from isofit.inversion.inverse import Inversion
from isofit.core.fileio import IO

In [3]:
import tkinter
import matplotlib
matplotlib.use('TkAgg')
%matplotlib inline

In [4]:
cluster_1 = np.loadtxt(r'C:\Users\vpatro\Desktop\Final_Spectra\Cl_1.txt')

In [5]:
cluster_1.shape

(568528, 321)

In [24]:
def sample_spectra(cluster_spectra:np.array):

    sample_sets = []
    for i in range(5):
        sample = random.sample(range(0, cluster_spectra.shape[0]), 10)
        sample_sets.append(cluster_spectra[sample])
        
    sample_sets = np.array(sample_sets)
        
    return sample_sets

In [30]:
def expand_spectra(cluster_spectra: np.array):
    display = np.empty((cluster_spectra.shape[0], cluster_spectra.shape[1], 425))
    display[:] = np.nan

    """
    for i in range(cluster_spectra.shape[0]):
        display[i, 3:182] = cluster_spectra[i][0:179]
        display[i, 224:276] = cluster_spectra[i][179:231]
        display[i, 329:419] = cluster_spectra[i][231:321]
        
    """
    
    display[:,:, 3:182] = cluster_spectra[:,:,0:179]
    display[:,:, 224:276] = cluster_spectra[:,:,179:231]
    display[:,:, 329:419] = cluster_spectra[:,:,231:321]
    
    
        
    return display
    

In [31]:
sample_sets_1 = sample_spectra(cluster_1)

In [32]:
sample_sets_1.shape

(5, 10, 321)

In [33]:
display_sample_sets_1 = expand_spectra(sample_sets_1)

In [34]:
display_sample_sets_1.shape

(5, 10, 425)

In [35]:
def run_ISOFIT(input_reflectance: np.array) -> np.array:
    
    # Surface model
    surface_model("C:\\Users\\vpatro\\Desktop\\isofit_vp\\examples\\20171108_Pasadena\\configs\\ang20171108t184227_surface.json")

    # Creating ISOFIT objects
    config = create_new_config("C:\\Users\\vpatro\\Desktop\\isofit_vp\\examples\\20171108_Pasadena\\configs\\ang20171108t184227_beckmanlawn.json")
    fm = ForwardModel(config)
    inv = Inversion(config, fm)
    io = IO(config, fm)
    io.get_components_at_index(0, 0)
    geom = io.current_input_data.geom # alternately, call via geom = Geometry()...this won't have data from the above config file
    meas = io.current_input_data.meas  # a numpy  array

    # adding in Radiative Transfer Parameters
    #x = np.loadtxt("C:\\Users\\vpatro\\Desktop\\avirisng_input.txt")
    input_reflectance = np.append(input_reflectance,1.75)
    input_reflectance = np.append(input_reflectance,0.05)
    
    radiance_measurement = fm.calc_rdn(input_reflectance,geom)
    OE_estimation = inv.invert(radiance_measurement, geom)[-1]
    
    return OE_estimation

In [ ]:
def get_residual(a:np.array, b:np.array):
    residual = a-b
    return residual

In [42]:
residuals = []
for i in range(10):
    input_vec = display_sample_sets_1[0][i]
    OE_estimation = run_ISOFIT(input_vec)
    residuals.append(residuals)

ERROR:root:MODTRAN directory not defined in config file.


SystemExit: MODTRAN directory not defined in config file.

C:\Users\vpatro\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3377: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
